In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import BatchLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GraphMasking
from spektral.data import Graph

In [4]:
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

In [5]:
dataset = QM9(amount=1000)

Loading QM9 dataset.
Reading SDF


100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 1099.39it/s]


In [6]:
for i in range(len(dataset)):
    dataset[i] = Graph(x=dataset[i].x, a=dataset[i].a, e=dataset[i].e, y=dataset[i].y[5:7])

In [7]:
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = 2     # Dimension of the target

In [8]:
idxs = np.random.permutation(len(dataset))
split = int(0.9 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]

In [9]:
class Net(Model):
    def __init__(self):
        super().__init__()
        self.masking = GraphMasking()
        self.conv1 = ECCConv(32, activation="relu")
        self.conv2 = ECCConv(32, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e = inputs
        x = self.masking(x)
        x = self.conv1([x, a, e])
        x = self.conv2([x, a, e])
        output = self.global_pool(x)
        output = self.dense(output)

        return output

In [10]:
model = Net()
optimizer = Adam(learning_rate)
model.compile(optimizer=optimizer, loss="mse")

In [11]:
loader_tr = BatchLoader(dataset_tr, batch_size=batch_size, mask=True)
model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs)

Epoch 1/10
29/29 [==============================] - 2s 23ms/step - loss: 1.4025
Epoch 2/10
29/29 [==============================] - 1s 24ms/step - loss: 0.1006
Epoch 3/10
29/29 [==============================] - 1s 23ms/step - loss: 0.0463
Epoch 4/10
29/29 [==============================] - 1s 23ms/step - loss: 0.0349
Epoch 5/10
29/29 [==============================] - 1s 22ms/step - loss: 0.0289
Epoch 6/10
29/29 [==============================] - 1s 23ms/step - loss: 0.0240
Epoch 7/10
29/29 [==============================] - 1s 23ms/step - loss: 0.0210
Epoch 8/10
29/29 [==============================] - 1s 23ms/step - loss: 0.0180
Epoch 9/10
29/29 [==============================] - 1s 24ms/step - loss: 0.0160
Epoch 10/10
29/29 [==============================] - 1s 24ms/step - loss: 0.0151


In [10]:
print("Testing model")
loader_te = BatchLoader(dataset_te, batch_size=batch_size, mask=True)
loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}".format(loss))

Testing model
4/4 [==============================] - 0s 12ms/step - loss: 0.0254
Done. Test loss: 0.025424746796488762
